In [288]:
# Библиотека для работы с HTTP-запросами. Будем использовать ее для обращения к API HH
import requests
 
# Пакет для удобной работы с данными в формате json
import json
 
# Модуль для работы со значением времени
import time
 
import pandas as pd
import numpy as np
from tqdm import tqdm

In [305]:
def getPage(page = 0):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        text - Текст фильтра
        area - Индекс страны или города. Например, 113-rus
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
        per_page - Кол-во вакансий на 1 странице
    """
     
    # Справочник для параметров GET-запроса
    params = {
        'text': f'NAME:c++',
        'area': 113,
        'page': page,
        'per_page': 100 
    }
     
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data
 
    
# Считываем первые 200 вакансий
jsObj = []
for page in tqdm(range(0, 2)):
    # Преобразуем текст ответа запроса в справочник Python
    js = json.loads(getPage(page))
    jsObj.append(js)
    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (js['pages'] - page) <= 1:
        break
     
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 2.5 сек мы может подождать
    time.sleep(0.25)
     
print('Страницы поиска собраны')

100%|██████████| 2/2 [00:02<00:00,  1.12s/it]

Страницы поиска собраны


In [306]:
# Создаем списки для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
#snippet = [] # Список описаний вакансий
salary = [] # Список зарплат

# Заполняем списки для таблиц
for i in tqdm(range(len(jsObj))):
    for j in range(len(jsObj[i]['items'])):
        IDs.append(jsObj[i]['items'][j]['id'])
        names.append(jsObj[i]['items'][j]['name'])
        snippet.append(jsObj[i]['items'][j]['snippet']['requirement'])
        try:
            salary.append(jsObj[i]['items'][j]['salary']['from'])
        except:
            salary.append(np.nan)

100%|██████████| 2/2 [00:00<00:00, 2307.73it/s]


In [307]:
#Создаем списки для столбцов таблицы skills
skills_name=[]
# Т.к. навыки хранятся в виде массива, то проходимся по нему циклом

for i in range(len(jsObj)):
    for j in tqdm(range(len(jsObj[i]['items']))):
        skills=str()
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req=requests.get(jsObj[i]['items'][j]['url'])
        data = req.content.decode()
        req.close()
        jsVac = json.loads(data)
        
        for skl in jsVac['key_skills']:
            skills = skills + skl['name']+','
        skills_name.append(skills[:-1])
        time.sleep(0.25)

100%|██████████| 100/100 [01:13<00:00,  1.36it/s]


In [308]:
pd.DataFrame({'ids':IDs,'names':names,'skills':skills_name,'salary':salary},index=IDs).to_csv('c++.csv', index=False)

# ----

In [413]:
df1 = pd.read_csv('./php.csv',index_col='ids')
df2 = pd.read_csv('./python.csv',index_col='ids')
df3 = pd.read_csv('./c++.csv',index_col='ids')

# Analysis

In [310]:
df1.describe() # php

,salary
count,160.000000
mean,122785.625000
std,60459.742671
min,400.000000
25%,80000.000000
50%,120000.000000
75%,150000.000000
max,300000.000000


In [311]:
df2.describe() # python

,salary
count,115.000000
mean,132378.260870
std,70221.191611
min,3000.000000
25%,80000.000000
50%,120000.000000
75%,180000.000000
max,300000.000000


In [312]:
df3.describe() # c++

,salary
count,97.000000
mean,153072.164948
std,80623.643066
min,4000.000000
25%,100000.000000
50%,150000.000000
75%,200000.000000
max,400000.000000


# Preprocessing

In [414]:
def preprocessing(df, name):
    df.skills = df.skills.str.lower().str.split(',')
    df = df.explode('skills').drop_duplicates()
    df['count'] = 1
    df = df.pivot(columns='skills',values='count').fillna(0)
    try:
        df = df.drop([''])
    except:
        pass
    df['language'] = name
    
    return df.loc[:, df.columns.notnull()]

In [415]:
df1 = preprocessing(df1, 1)#'php'
df2 = preprocessing(df2, 2)#'python'
df3 = preprocessing(df3, 3)#'c++'

In [404]:
df1.sum().sort_values(ascending=False).dropna()[:20]

skills
php           127.0
mysql         104.0
git            81.0
javascript     60.0
ооп            58.0
postgresql     44.0
laravel        41.0
html           39.0
symfony        35.0
sql            34.0
linux          33.0
css            33.0
jquery         32.0
yii            28.0
php5           23.0
redis          22.0
docker         21.0
php7           19.0
rabbitmq       14.0
ajax           14.0
dtype: float64

In [405]:
df2.sum().sort_values(ascending=False)[:20]

skills
python              133.0
postgresql           67.0
git                  66.0
django framework     64.0
linux                51.0
sql                  44.0
javascript           22.0
docker               20.0
redis                18.0
ооп                  18.0
mysql                16.0
rest                 15.0
rabbitmq             15.0
flask                11.0
базы данных          10.0
английский язык      10.0
mongodb               8.0
celery                7.0
atlassian jira        7.0
fastapi               6.0
dtype: float64

In [406]:
df3.sum().sort_values(ascending=False)[:20]

skills
c++                 117.0
linux                61.0
git                  45.0
c/c++                39.0
qt                   37.0
ооп                  34.0
python               23.0
stl                  23.0
c#                   19.0
boost                12.0
sql                  12.0
разработка по         9.0
postgresql            8.0
tcp/ip                8.0
java                  7.0
английский язык       7.0
ms visual studio      7.0
unity                 6.0
opengl                5.0
arm                   5.0
dtype: float64

In [416]:
data = pd.concat([df1,df2,df3]).fillna(0)

# ML

In [417]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [418]:
y = pd.get_dummies(data.language)

In [419]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy_score(y_test, pred)

0.9823008849557522

In [420]:
clf.predict_proba(X_test)

[array([[0.98, 0.02],
        [0.86, 0.14],
        [0.99, 0.01],
        [0.  , 1.  ],
        [0.95, 0.05],
        [0.72, 0.28],
        [0.98, 0.02],
        [1.  , 0.  ],
        [0.07, 0.93],
        [1.  , 0.  ],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [0.02, 0.98],
        [0.06, 0.94],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [0.01, 0.99],
        [0.99, 0.01],
        [0.99, 0.01],
        [1.  , 0.  ],
        [0.01, 0.99],
        [0.31, 0.69],
        [0.01, 0.99],
        [0.99, 0.01],
        [0.98, 0.02],
        [0.7 , 0.3 ],
        [0.05, 0.95],
        [0.98, 0.02],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [0.21, 0.79],
        [0.08, 0.92],
        [0.98, 0.02],
        [0.97, 0.03],
        [0.69, 0.31],
        [0.96, 0.04],
        [0.02, 0.98],
        [0.04, 0.96],
        [1.  , 0.  ],
        [0.02, 0.98],
        [1.  , 0.  ],
        [1